# 1. Introducción

En el siguiente ejercicio se realizará el **producto escalar** entre dos vectores de igual dimensiones.

El algoritmo está basado en la función **dot** nivel 1 [1], de la biblioteca **BLAS** [2] que resuelve la ecuación:
<center>$res = \sum_{i-1}^{n} X_i * Y_i$</center>

La idea principal es mostrar la perfomance del funcionamiento de **estructuras de una dimensión** para gran cantidad de elementos.

Se utilizará Python con las prestaciones brindadas por CUDA para el uso de tecnología GPGPU.

En este último aspecto, se implementaron **dos funciones kernel** que se encargan de la **multiplicación** de los componentes de los vectores y la **suma** de esos resultados parciales, respectivamente.

---
# 2. Armado del ambiente

### Instalando el **módulo CUDA** de Python

In [ ]:
!pip install pycuda

### Importación de **bibliotecas**

In [ ]:
# Importamos el driver y el compilador para CUDA
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Importamos bibliotecas estándares de Python
from datetime import datetime
import numpy
import math

---
# 3. Desarrollo

In [ ]:
#@title ### 3.1. Parámetros de ejecución {vertical-output: true}
#@markdown ---
#@markdown Cantidad de elementos de los arrays X e Y:
cant_elementos =   5000000#@param {type: "number"}
#@markdown ---


# Definición de función que transforma el tiempo en milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# Capturamos el tiempo inicial
tiempo_total = datetime.now()

# CPU - Defino los vectores y el resultado en CPU
x_cpu = numpy.random.randint(1, 10, cant_elementos)
x_cpu = x_cpu.astype(numpy.int64())
y_cpu = numpy.random.randint(1,10, cant_elementos)
y_cpu = y_cpu.astype(numpy.int64())
res_cpu = numpy.empty_like(numpy.int64())

# CPU - Reservo la memoria GPU en base a los vectores creados
x_gpu = cuda.mem_alloc(x_cpu.nbytes)
y_gpu = cuda.mem_alloc(y_cpu.nbytes)
res_gpu = cuda.mem_alloc(res_cpu.nbytes)

# GPU - Copio la memoria desde CPU a GPU
cuda.memcpy_htod(x_gpu, x_cpu)
cuda.memcpy_htod(y_gpu, y_cpu)
cuda.memcpy_htod(res_gpu, res_cpu)

# CPU - Defino la función kernel que multiplica los elementos
module_mul = SourceModule("""
__global__ void kernel_dot_mul(int n, int *X, int *Y) {
  
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if(idx < n) {
    Y[idx]  = X[idx] * Y[idx];
  }

}
""")

# CPU - Defino la función kernel que suma los elementos
module_sum = SourceModule("""
__global__ void kernel_dot_sum(int n, int *Y) {
  
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if(2*idx + 1 < n) {
    Y[idx] = Y[2*idx] + Y[2*idx + 1];
  }
  else if(2*idx + 1 == n) {
    Y[0] += Y[2*idx];
  }

}
""")

# CPU - Obtenemos la funciones kernel creadas
kernel_mul = module_mul.get_function("kernel_dot_mul")
kernel_sum = module_sum.get_function("kernel_dot_sum")

# Capturamos el tiempo inicial de GPU
tiempo_gpu = datetime.now()

# GPU - Definimos la dimensión de threads y bloques
dim_hilo = 256
dim_bloque = numpy.int((cant_elementos + dim_hilo - 1) / dim_hilo)

# GPU - Ejecuta el kernel para la multiplicación
kernel_mul(numpy.int64(cant_elementos), 
           x_gpu, y_gpu, 
           block=(dim_hilo, 1, 1), grid=(dim_bloque, 1, 1))

# GPU - Ejecuta el kernel para las sumas
n = cant_elementos
while n/2 >= 1:
  kernel_sum(numpy.int64(n), 
             y_gpu, 
             block=(dim_hilo, 1, 1), grid=(dim_bloque, 1, 1))
  n = math.floor(n/2)

# Capturamos el tiempo total que tardó GPU
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copiamos el resultado desde la memoria GPU a CPU
cuda.memcpy_dtoh(res_cpu, y_gpu)

# Capturamos el tiempo total de todo el ejercicio
tiempo_total = datetime.now() - tiempo_total

# Mostramos el resultado del proceso
print("Cantidad de elementos en los arrays: ", cant_elementos)
print("Dim. Thread x: ", dim_hilo, " - Dim. Bloque x:", dim_bloque, "\n")
print("Tiempo total: ", tiempo_en_ms(tiempo_total), "[ms]")
print("Tiempo GPU: ", tiempo_en_ms(tiempo_gpu), "[ms]\n")
print("Resultado de DOT: ", res_cpu)

---
# 4. Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  pip install pycuda    | Instala el módulo de CUDA para Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  datetime.now()        | Toma el tiempo inicial del ejercicio.
CPU      |  numpy.random.randint  | Inicializa los vectores X e Y con enteros aleatorios.
CPU      | astype                 | Castea los vectores a un tipo de dato especificado.
CPU      | numpy.empty_like       | Retorna un array con la misma forma y tipo que el especificado.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia la memoria desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código de las funciones kernel. 
CPU      |  module.get_function() | Obtenemos las funciones del kernel GPU.
CPU      |  datetime.now()        | Toma el tiempo inicial de GPU.
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel_mul y kernel_sum              | Ejecuta el kernel en GPU
CPU      |  datetime.now()        | Toma el tiempo total de GPU.
**GPU**      |  cuda.memcpy_dtoh( )   | Copia el resultado desde memoria GPU a memoria CPU.
CPU      |  datetime.now()        | Toma el tiempo total del ejercicio.
CPU      |  print()               | Informa los resultados.



---
# 5. Conclusiones



---
# 6. Bibliografia

[1] Función DOT de biblioteca BLAS: [Página Web](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-dot.html)

[2] Biblioteca BLAS: [BLAS](http://www.netlib.org/blas/)


